In [3]:
import pymysql
import pandas as pd
import datetime
from datetime import date

## low value로 하면 안됨 전날 종가로 해야함

conn = pymysql.connect(
*
)

# 당일 데이터 BOX TABLE 추가

def insert_box_table(data,new_data,new_cols,today) :
    
    for li in new_data :
        
        nesting_order = 0
        
        for key in list(box_table_stock.keys()) :
        
            if key == li[new_cols['stockName']] :
                
                nesting_order = data[key] + 1
            
        threshold = li[new_cols['val_close']] - li[new_cols['comparePrev']]*0.45
        
        sql_execute(
            f"""
                insert into boxstock values ('{li[new_cols['stockName']]}',{nesting_order}, {li[new_cols['val_close']]},
                {li[new_cols['val_open']]},{li[new_cols['val_close']]},{li[new_cols['val_close']]},'F',
                {threshold},0,'N','{today}')
            """
        )
    print('new data 추가')
    


# 당일 데이터 BOX TABLE 업데이트
def update_box_table(data,cols) :   
    
    for li in data :
        
        if li[cols['type_code']] == 'N' :
            
            if li[cols['val_high']] < li[cols['t_close']] :
                
                sql_execute(
                    f"""
                        update boxStock set val_high = {li[cols['t_close']]} where stockname = '{li[cols['stockName']]}'
                        and nesting_order = {li[cols['nesting_order']]}
                    """
                )
                
            elif li[cols['val_low']] > li[cols['t_close']] :
                
                  sql_execute(
                    f"""
                        update boxStock set val_low = {li[cols['t_close']]} where stockname = '{li[cols['stockName']]}'
                        and nesting_order = {li[cols['nesting_order']]}
                    """
                )

        

    sql_execute(
                f"""
                    update boxStock set box_day_counting = box_day_counting + 1 where type_code = 'N'
                """
                )
            
    print('box table update')
    
    
# 게단 상태 업데이트
def update_is_stair(data,cols) : 
    
    for li in data :
        
        if li[cols['type_code']] == 'N' :                    
            
            if li[cols['rate']] <= 7 and (li[cols['t_close']] > li[cols['val_high']]*0.8 ):
                
                if li[cols['isStair']] == 'F':
                
                    sql_execute(
                    f"""
                        update boxStock set isStair = 'T' where stockname = '{li[cols['stockName']]}' and 
                        nesting_order =  {li[cols['nesting_order']]}

                    """
                    )
            elif li[cols['t_close']] < (li[cols['val_high']]*0.8) :
                
                if li[cols['isStair']] == 'T':
                    
                    sql_execute(
                    f"""
                        update boxStock set isStair = 'F' where stockname = '{li[cols['stockName']]}' and 
                        nesting_order =  {li[cols['nesting_order']]}

                    """
                    )
    print('계단 여부 체크')


# jump 상태 업데이트
def update_jump_stock(data,cols):
    
    for li in data :
        
        if li[cols['type_code']] == 'N' :   
        
            if li[cols['rate']] > 7 and  li[cols['t_close']] > li[cols['val_high']]:
                
                sql_execute(
                f"""
                    update boxStock set type_code = 'U' where stockname = '{li[cols['stockName']]}' and 
                    nesting_order =  {li[cols['nesting_order']]}
                    
                """
                )
    print('급등 종목 상태 업데이트')         

# leave table 이탈, 복구 체크
def check_leave_stock(data,cols):
    
    for li in data:
        
        if li[cols['type_code']] == 'N' :
        
            if li[cols['threshold']] > li[cols['t_close']] :

                sql_execute(
                    f"""
                        update boxStock set type_code = 'D' where stockname = '{li[cols['stockName']]}' and 
                        nesting_order =  {li[cols['nesting_order']]}
                    """
                )

                sql_execute(
                    f"""
                        insert into leaveBox values ('{li[cols['stockName']]}',{li[cols['nesting_order']]},'N')
                    """
                )
            
        elif li[cols['type_code']] == 'D' :
            
            if li[cols['threshold']] < li[cols['t_close']] :
             
                sql_execute(
                    f"""
                       delete from leaveBox where stockname = '{li[cols['stockName']]}' and nesting_order =  {li[cols['nesting_order']]}
                    """
                )
                
                sql_execute(
                    f"""
                         update boxStock set type_code = 'N' where stockname = '{li[cols['stockName']]}' and 
                        nesting_order =  {li[cols['nesting_order']]}
                    """
                )
    print('threhold 이탈, 복구 체크')
            


    
# open_value 이탈 값들 삭제 휴지통으로 이동 

def delete_leave_table(data,cols,today):
    
    for li in data:
        
        if li[cols['first_open_value']] > li[cols['t_close']]:
            
            if li[cols['nesting_order']] > 0 :
                
                sql_execute(
                    f"""
                        update boxStock set type_code = 'N' where stockname = '{li[cols['stockName']]}' and 
                        nesting_order = {li[cols['nesting_order']]-1}
                    """
                )
            
            sql_execute(
                f"""
                insert into deletedBoxTable values ('{li[cols['stockName']]}','{today}')
                """
            )
            sql_execute(
                f"""
                delete from boxStock where stockName = '{li[cols['stockName']]}' and nesting_order = {li[cols['nesting_order']]}
                """)
            
            del data[data.index(li)]
            
    print('open_value 이탈 체크 후 삭제')
    
def sql_execute(sql):
    
    cursor = conn.cursor()
    
    cursor.execute(sql)
    
    conn.commit()
    
def get_col(col_list):
    
    output = {}
    
    for i in range(len(col_list)):
        
        output[col_list[i][0]] = i
        
    return output

def get_max_nesting_order(nesting_list):
    
    output = {}
    
    for i in nesting_list:
        
        output[i[0]] = i[1]
        
    return output
    
    
if __name__ == '__main__':
    
    start = datetime.datetime.strptime('20220104','%Y%m%d')
    end = datetime.datetime.strptime('20220503','%Y%m%d')

    for i in range((end-start).days+1) :
        
#         today = '2021-05-10'
        
        today = (start+datetime.timedelta(days=i)).strftime("%Y-%m-%d")
        
        print(today,' 실행 중 ...')
        print('==============\n')

        sql = f"""
            select * from boxStock
                    join (select stockName ,val_close as t_close,rate from dailyinfo where stock_date = '{today}') as info
                on boxStock.stockName = info.stockName"""

        cursor = conn.cursor()

        cursor.execute(sql)

        cols = get_col(cursor.description)

        result = list(cursor.fetchall())
        
        if not result : 
            
            continue

        ### main

        delete_leave_table(result,cols,today)
        check_leave_stock(result,cols)
        update_jump_stock(result,cols)
        update_is_stair(result,cols)
        update_box_table(result,cols)



        ### new box table update

        sql = f"""
            select * from dailyinfo where stock_date = '{today}' and rate > 7 and vol > 100000;
        """

        cursor = conn.cursor()

        cursor.execute(sql)

        new_cols = get_col(cursor.description)

        new_data = list(cursor.fetchall())

        cursor = conn.cursor()


        sql = """
            select stockName, max(nesting_order) as max_nesting_order from boxstock group by stockName;
        """

        cursor.execute(sql)

        box_table_stock = get_max_nesting_order(list(cursor.fetchall()))


        insert_box_table(box_table_stock, new_data,new_cols,today)
        
        print()
        print('==============\n')

conn.close()
    

2022-01-04  실행 중 ...

open_value 이탈 체크 후 삭제
threhold 이탈, 복구 체크
급등 종목 상태 업데이트
계단 여부 체크
box table update
new data 추가


2022-01-05  실행 중 ...

open_value 이탈 체크 후 삭제
threhold 이탈, 복구 체크
급등 종목 상태 업데이트
계단 여부 체크
box table update
new data 추가


2022-01-06  실행 중 ...

open_value 이탈 체크 후 삭제
threhold 이탈, 복구 체크
급등 종목 상태 업데이트
계단 여부 체크
box table update
new data 추가


2022-01-07  실행 중 ...

open_value 이탈 체크 후 삭제
threhold 이탈, 복구 체크
급등 종목 상태 업데이트
계단 여부 체크
box table update
new data 추가


2022-01-08  실행 중 ...

2022-01-09  실행 중 ...

2022-01-10  실행 중 ...

open_value 이탈 체크 후 삭제
threhold 이탈, 복구 체크
급등 종목 상태 업데이트
계단 여부 체크
box table update
new data 추가


2022-01-11  실행 중 ...

open_value 이탈 체크 후 삭제
threhold 이탈, 복구 체크
급등 종목 상태 업데이트
계단 여부 체크
box table update
new data 추가


2022-01-12  실행 중 ...

open_value 이탈 체크 후 삭제
threhold 이탈, 복구 체크
급등 종목 상태 업데이트
계단 여부 체크
box table update
new data 추가


2022-01-13  실행 중 ...

open_value 이탈 체크 후 삭제
threhold 이탈, 복구 체크
급등 종목 상태 업데이트
계단 여부 체크
box table update
new data 추가


2022-01-14  실행 중 ...

2022-03-21  실행 중 ...

open_value 이탈 체크 후 삭제
threhold 이탈, 복구 체크
급등 종목 상태 업데이트
계단 여부 체크
box table update
new data 추가


2022-03-22  실행 중 ...

open_value 이탈 체크 후 삭제
threhold 이탈, 복구 체크
급등 종목 상태 업데이트
계단 여부 체크
box table update
new data 추가


2022-03-23  실행 중 ...

open_value 이탈 체크 후 삭제
threhold 이탈, 복구 체크
급등 종목 상태 업데이트
계단 여부 체크
box table update
new data 추가


2022-03-24  실행 중 ...

open_value 이탈 체크 후 삭제
threhold 이탈, 복구 체크
급등 종목 상태 업데이트
계단 여부 체크
box table update
new data 추가


2022-03-25  실행 중 ...

open_value 이탈 체크 후 삭제
threhold 이탈, 복구 체크
급등 종목 상태 업데이트
계단 여부 체크
box table update
new data 추가


2022-03-26  실행 중 ...

2022-03-27  실행 중 ...

2022-03-28  실행 중 ...

open_value 이탈 체크 후 삭제
threhold 이탈, 복구 체크
급등 종목 상태 업데이트
계단 여부 체크
box table update
new data 추가


2022-03-29  실행 중 ...

open_value 이탈 체크 후 삭제
threhold 이탈, 복구 체크
급등 종목 상태 업데이트
계단 여부 체크
box table update
new data 추가


2022-03-30  실행 중 ...

open_value 이탈 체크 후 삭제
threhold 이탈, 복구 체크
급등 종목 상태 업데이트
계단 여부 체크
box table update
new data 추가


2022-03-31  실행 중 ...